In [2]:
import requests
api_url = 'http://poppy.local:8080'

In [3]:
resource_uri = '/robot.json'
response = requests.get(api_url + resource_uri)
response.json()

{'motors': [{'registers': ['registers',
    'goal_speed',
    'compliant',
    'safe_compliant',
    'angle_limit',
    'id',
    'name',
    'model',
    'present_position',
    'goal_position',
    'present_speed',
    'moving_speed',
    'present_load',
    'torque_limit',
    'lower_limit',
    'upper_limit',
    'present_voltage',
    'present_temperature',
    'pid',
    'led',
    'control_mode'],
   'goal_speed': -15.984000000000002,
   'compliant': True,
   'angle_limit': [-150.0, 150.0],
   'id': 1,
   'name': 'm1',
   'model': 'XL-320',
   'present_position': -62.02,
   'goal_position': -14.52,
   'present_speed': 0.0,
   'moving_speed': 15.984000000000002,
   'present_load': 0.0,
   'torque_limit': 70.0,
   'lower_limit': -150.0,
   'upper_limit': 150.0,
   'present_voltage': 7.0,
   'present_temperature': 29.0,
   'pid': [4, 2, 0],
   'led': 'off',
   'control_mode': 0},
  {'registers': ['registers',
    'goal_speed',
    'compliant',
    'safe_compliant',
    'angle_limit

In [10]:
def get_motor_names() -> list:
    motor_list_uri = '/motors/list.json'
    res = requests.get(api_url + motor_list_uri)
    res = res.json()
    
    return res['motors']

In [11]:
# getting registers from motor name
motor_name = 'm1'
motor_registers_uri = f'/motors/{motor_name}/registers/list.json'
res = requests.get(api_url + motor_registers_uri)
res = res.json()
res

{'registers': ['registers',
  'goal_speed',
  'compliant',
  'safe_compliant',
  'angle_limit',
  'id',
  'name',
  'model',
  'present_position',
  'goal_position',
  'present_speed',
  'moving_speed',
  'present_load',
  'torque_limit',
  'lower_limit',
  'upper_limit',
  'present_voltage',
  'present_temperature',
  'pid',
  'led',
  'control_mode']}

In [49]:
def get_joint_angles():
    register = 'present_position'
    motor_names = get_motor_names()
    joint_angles_uri = f'/motors/registers/{register}/list.json'
    response = requests.get(api_url + joint_angles_uri)
    response = response.json()
    
    joint_angles = []
    for mname in motor_names:
        joint_angles.append(response[register][mname])
    
    return joint_angles

get_joint_angles()
        

[-62.02, -24.19, -18.62, -48.83, -1.91]

In [22]:
def get_joint_limits():
    motor_names = get_motor_names()
    
    reg_low = 'lower_limit'
    reg_up = 'upper_limit'
    
    low_limits_uri = f'/motors/registers/{reg_low}/list.json'
    up_limits_uri = f'/motors/registers/{reg_up}/list.json'
    
    res = requests.get(api_url + low_limits_uri)
    lower_limits = res.json()
    print(lower_limits)
    
    res = requests.get(api_url + up_limits_uri)
    upper_limits = res.json()
    print(upper_limits)
    
    joint_limits = {}
    for mname in motor_names:
        lower_limit = lower_limits['lower_limit'][mname]
        upper_limit = upper_limits['upper_limit'][mname]
        joint_limits[mname] = [lower_limit, upper_limit]
    
    return joint_limits

joint_limits = get_joint_limits()
print(joint_limits)

{'lower_limit': {'m1': -150.0, 'm2': 89.88, 'm3': 89.88, 'm4': -150.0, 'm5': 89.88}}
{'upper_limit': {'m1': 150.0, 'm2': -125.07, 'm3': -89.88, 'm4': 150.0, 'm5': -89.88}}
{'m1': [-150.0, 150.0], 'm2': [89.88, -125.07], 'm3': [89.88, -89.88], 'm4': [-150.0, 150.0], 'm5': [89.88, -89.88]}


In [27]:
def calculate_wait_time(angles, speed=180):
        curr_pos = get_joint_angles()
        motor_names = get_motor_names()
        diffs = [abs(angles[i] - curr_pos[i]) for i in range(len(motor_names))]
        
        max_speed = 360
        if speed > max_speed:
            speed = max_speed
        return 1/(speed/max(diffs))

In [53]:
def set_joint_angles(angles: list, speed=180):
    move_joint_uri = '/motors/goto.json'
    
    motor_names = get_motor_names()
    move_command = {}
    move_command['motors'] = motor_names
    move_command['positions'] = angles
    
    wait_time = calculate_wait_time(angles, speed=speed)
    move_command['duration'] = str(wait_time)
    move_command['wait'] = True
    
    res = requests.post(api_url + move_joint_uri, json=move_command)
    print(res.json())
    
angles = [0, 0, 0, 0, 0]
set_joint_angles(angles)

{'motors': {'m1': 0.0, 'm2': 0.0, 'm3': 0.0, 'm4': 0.0, 'm5': 0.0}, 'duration': 0.41300000000000003, 'waiting': True}


In [40]:
move_command = {
    "motors": ["m1"],
    "positions": [20],
    "duration": "2",
    "wait": False
}

move_joint_uri = '/motors/goto.json'
res = requests.post(api_url + move_joint_uri, json=move_command)
print(res.json())


{'motors': {'m1': 20.0}, 'duration': 2.0, 'waiting': False}


In [50]:
def set_motor_led(motor_name, led_state):
    led_uri = f'/motors/{motor_name}/registers/led/value.json'
    
    
    
    res = requests.post(api_url + led_uri, json=led_state)
    print(res.json())

set_motor_led('m1', 'red')



{'led': 'red'}


In [51]:
def set_motor_compliance(motor_name, isCompliant):
    comp_uri = f'/motors/{motor_name}/registers/compliant/value.json'
    
    res = requests.post(api_url + comp_uri, json=isCompliant)
    print(res.json())

set_motor_compliance('m1', False)
    

{'compliant': False}


In [52]:
def set_motors_compliance(compliance_states):
    motor_names = get_motor_names()
    for i in range(len(motor_names)):
        set_motor_compliance(motor_names[i], compliance_states[i]) 

set_motors_compliance([False for i in range(5)])

{'compliant': False}
{'compliant': False}
{'compliant': False}
{'compliant': False}
{'compliant': False}
